<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
#default_exp models.base

In [ ]:
#export
import torch
from torch import nn

class TTSModel(nn.Module):
    def infer(self):
        raise NotImplemented

    def forward(self):
        raise NotImplemented

    def from_pretrained(self,checkpoint_path,device, ignore_layers = None):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        if "state_dict" in checkpoint.keys():
            model_dict = checkpoint["state_dict"]
            if ignore_layers:
                model_dict = {k: v for k, v in model_dict.items()
                              if k not in ignore_layers}
                dummy_dict = self.state_dict()
                dummy_dict.update(model_dict)
                model_dict = dummy_dict
            self.load_state_dict(model_dict)
        if device == 'cuda:0':
            self.cuda()
        
    def to_checkpoint(self):
        return dict(model=self.state_dict())

    @classmethod
    def create(cls, name, opts, folders, all_speakers = True):
        pass
#         model_cls = cls.get_class(name)
#         folders = pd.read_csv(folders)
#         for folder in folders:


#         return model_cls(opts)